In [11]:
import numpy as np
from typing import List, Tuple, Callable, Union
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.sat.python import cp_model


def maximize_accuracy(class_matrix: np.ndarray) -> np.ndarray:
    """
    Maximize number of correct predictions: sum(class_matrix[i, x[i]] for i in range(N))

    Parameters
    ----------
    class_matrix : np.ndarray
        2D Numpy array (C, C) of the number of points in each cluster
        Element (i, j) is the number of points in cluster i of actual label j

    Returns
    -------
    np.ndarray | None
        1D Numpy array of the optimized mapping from predicted cluster index to actual cluster index
        If problem is feasible, return the optimized mapping
        If the optimization fails, return None
    """
    model = cp_model.CpModel()
    C = class_matrix.shape[0]
    x = [[model.NewIntVar(0, 1, f'x[{i},{j}]') for j in range(C)] for i in range(C)]

    for i in range(C):
        model.Add(sum(x[i][j] for j in range(C)) == 1)
        model.Add(sum(x[j][i] for j in range(C)) == 1)

    model.Maximize(sum(class_matrix[i, j] * x[i][j] for i in range(C) for j in range(C)))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        result = np.full(C, -1, dtype=int)
        for i in range(C):
            for j in range(C):
                if solver.Value(x[i][j]) == 1:
                    result[i] = j
        return result
    elif status == cp_model.FEASIBLE:
        result = np.full(C, -1, dtype=int)
        for i in range(C):
            for j in range(C):
                if solver.Value(x[i][j]) == 1:
                    result[i] = j
    else:
        return None

In [12]:
c = np.array([
    [125, 256, 332, 120, 15],
    [589, 235, 1258, 25, 16],
    [269, 196, 456, 222, 168],
    [125, 256, 333, 120, 125],
    [1250, 256, 332, 120, 15]
])
result = maximize_accuracy(c)
print(result)

[1 2 3 4 0]
